# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
from scipy.stats import linregress
import time
import requests
import json
import random

from citipy import citipy

from api_keys import weather_api_key

base_url = "http://api.openweathermap.org/data/2.5/weather?q="

## Generate Cities List

In [2]:
coordinates = []
cities = []

longitude = np.random.uniform(-180, 180, size=1500)
latitude = np.random.uniform(-90, 90, size=1500)

coordinates = zip(latitude, longitude)

for coordinate in coordinates:
    city_coordinates = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    if city_coordinates not in cities:
        cities.append(city_coordinates)
        
print(len(cities))

577


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [6]:
record_count = 0
set_count = 0

city_name = []
lat = []
lng = []
max_temp = []
humidity = []
clouds = []
wind_speed = []
country = []
date = []

print("-----------------------------")
print("Beginning Data Retrieval")
print("-----------------------------")

for city in cities:
    
    time.sleep(1.25)
    
    query_url = f"{base_url}{city}&units=imperial&appid={weather_api_key}"

    weather_response = requests.get(query_url)
    weather_json = weather_response.json()
    
    try:
        set_count = 1 + math.floor((record_count/50))
        record_count = record_count + 1
        
        city_name.append(weather_json["name"])
        lat.append(weather_json["coord"]["lat"])
        lng.append(weather_json["coord"]["lon"])
        max_temp.append(weather_json["main"]["temp_max"])
        humidity.append(weather_json["main"]["humidity"])
        clouds.append(weather_json["clouds"]["all"])
        wind_speed.append(weather_json["wind"]["speed"])
        country.append(weather_json["sys"]["country"])
        date.append(weather_json["dt"])
        
        print("Processing Record " + str(record_count) + " of Set " + str(set_count) + " | " + city)
        
    except:
        print("The city of " + city + " not found. Skipping...")
        
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

-----------------------------
Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | mataura
Processing Record 2 of Set 1 | port-cartier
Processing Record 3 of Set 1 | castro
The city of taolanaro not found. Skipping...
The city of illoqqortoormiut not found. Skipping...
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | mar del plata
Processing Record 8 of Set 1 | aklavik
Processing Record 9 of Set 1 | rikitea
Processing Record 10 of Set 1 | ilulissat
Processing Record 11 of Set 1 | mezen
Processing Record 12 of Set 1 | tananger


KeyboardInterrupt: 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_df = pd.DataFrame({'City':city_name, 'Latitude':lat, 'Longitude': lng, 'Max Temperature': max_temp,
                          'Humidity':humidity, 'Cloudiness': clouds, 'Wind Speed': wind_speed, 'Country':country, "Date":date})

weather_df[["Humidity", "Cloudiness"]] = weather_df[["Humidity", "Cloudiness"]].apply(pd.to_numeric)

weather_df.to_csv(r'C:\Users\kylie\OneDrive\Desktop\uofm-stp-data-pt-09-2020-u-c\02-Homework\06-Python-APIs\python-api-challenge\Weatherpy Documents\weather_df.csv')

weather_df

In [ ]:
weather_df.describe()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
high_humidity = weather_df.loc[weather_df["Humidity"] > 100]
high_humidity

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(weather_df["Latitude"], weather_df["Max Temperature"], color="lightcoral", edgecolor="maroon", alpha=0.75)
plt.title("City Latitude vs. Max Temperature (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.grid()
plt.savefig("Weatherpy Documents/City Latitude vs Max Temperature.png")
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(weather_df["Latitude"], weather_df["Humidity"], color="cadetblue", edgecolor="darkslategray", alpha=0.75)
plt.title("City Latitude vs. Humidity (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid()
plt.savefig("Weatherpy Documents/City Latitude vs Humidity.png")
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(weather_df["Latitude"], weather_df["Cloudiness"], color="teal", edgecolor="darkblue", alpha=0.75)
plt.title("City Latitude vs. Cloudiness (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid()
plt.savefig("Weatherpy Documents/City Latitude vs Cloudiness.png")
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(weather_df["Latitude"], weather_df["Wind Speed"], color="forestgreen", edgecolor="darkgreen", alpha=0.75)
plt.title("City Latitude vs. Wind Speed (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid()
plt.savefig("Weatherpy Documents/City Latitude vs Wind Speed.png")
plt.show()

## Linear Regression

In [ ]:
# Dataframe for Northern Hemisphere (greater than or equal to 0 degrees latitude)
north_df = weather_df.loc[weather_df["Latitude"] >= 0]

# Dataframe for Southern Hemisphere (less than 0 degrees latitude)
south_df = weather_df.loc[weather_df["Latitude"] < 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
plt.scatter(north_df["Latitude"], north_df["Max Temperature"], color="lightcoral", edgecolor="maroon", alpha=0.75)

x_values = north_df["Latitude"]
y_values = north_df["Max Temperature"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="maroon")
plt.annotate(line_eq,(45,25),fontsize=15,color="maroon")

plt.title("Northern Hemisphere \n City Latitude vs. Max Temperature (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")

print("The r value is: " + str(rvalue))
plt.savefig("Weatherpy Documents/Northern Hemisphere City Latitude vs Max Temperature.png")
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
plt.scatter(south_df["Latitude"], south_df["Max Temperature"], color="lightcoral", edgecolor="maroon", alpha=0.75)

x_values = south_df["Latitude"]
y_values = south_df["Max Temperature"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="maroon")
plt.annotate(line_eq,(-45,40),fontsize=15,color="maroon")

plt.title("Southern Hemisphere \n City Latitude vs. Max Temperature (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")

print("The r value is: " + str(rvalue))
plt.savefig("Weatherpy Documents/Southern Hemisphere City Latitude vs Max Temperature.png")
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(north_df["Latitude"], north_df["Humidity"], color="cadetblue", edgecolor="darkslategray", alpha=0.75)

x_values = north_df["Latitude"]
y_values = north_df["Humidity"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="darkslategray")
plt.annotate(line_eq,(25,90),fontsize=15,color="darkslategray")

plt.title("Northern Hemisphere \n City Latitude vs. Humidity (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

print("The r value is: " + str(rvalue))
plt.savefig("Weatherpy Documents/Northern Hemisphere City Latitude vs Humidity.png")
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(south_df["Latitude"], south_df["Humidity"], color="cadetblue", edgecolor="darkslategray", alpha=0.75)

x_values = south_df["Latitude"]
y_values = south_df["Humidity"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="darkslategray")
plt.annotate(line_eq,(-55,60),fontsize=15,color="darkslategray")

plt.title("Southern Hemisphere \n City Latitude vs. Humidity (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

print("The r value is: " + str(rvalue))
plt.savefig("Weatherpy Documents/Southern Hemisphere City Latitude vs Humidity.png")
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(north_df["Latitude"], north_df["Cloudiness"], color="teal", edgecolor="darkblue", alpha=0.75)

x_values = north_df["Latitude"]
y_values = north_df["Cloudiness"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="darkblue")
plt.annotate(line_eq,(25,80),fontsize=15,color="darkblue")

plt.title("Northern Hemisphere \n City Latitude vs. Cloudiness (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

print("The r value is: " + str(rvalue))
plt.savefig("Weatherpy Documents/Northern Hemisphere City Latitude vs Cloudiness.png")
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
plt.scatter(south_df["Latitude"], south_df["Cloudiness"], color="teal", edgecolor="darkblue", alpha=0.75)

x_values = south_df["Latitude"]
y_values = south_df["Cloudiness"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="darkblue")
plt.annotate(line_eq,(-55,20),fontsize=15,color="darkblue")

plt.title("Southern Hemisphere \n City Latitude vs. Cloudiness (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

print("The r value is: " + str(rvalue))
plt.savefig("Weatherpy Documents/Southern Hemisphere City Latitude vs Cloudiness.png")
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
plt.scatter(north_df["Latitude"], north_df["Wind Speed"], color="forestgreen", edgecolor="darkgreen", alpha=0.75)

x_values = north_df["Latitude"]
y_values = north_df["Wind Speed"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="darkgreen")
plt.annotate(line_eq,(25,10),fontsize=15,color="darkgreen")

plt.title("Northern Hemisphere \n City Latitude vs. Wind Speed (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

print("The r value is: " + str(rvalue))
plt.savefig("Weatherpy Documents/Northern Hemisphere City Latitude vs Wind Speed.png")
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
plt.scatter(south_df["Latitude"], south_df["Wind Speed"], color="forestgreen", edgecolor="darkgreen", alpha=0.75)

x_values = south_df["Latitude"]
y_values = south_df["Wind Speed"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="darkgreen")
plt.annotate(line_eq,(-55,5),fontsize=15,color="darkgreen")

plt.title("Southern Hemisphere \n City Latitude vs. Wind Speed (11/16/20)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

print("The r value is: " + str(rvalue))
plt.savefig("Weatherpy Documents/Southern Hemisphere City Latitude vs Wind Speed.png")
plt.show()